In [1]:
import os
import dill
import numpy as np
from fastai.tabular.all import *
from fastai.vision.all import *
from fastai.vision.utils import get_image_files
from Ambrosia import pre_process_image
from huggingface_hub import from_pretrained_fastai, push_to_hub_fastai
import gradio as gr
###########
# use this to laod on windows
import pathlib
temp = pathlib.PosixPath
pathlib.PosixPath = pathlib.WindowsPath
###########

# from PIL import Image

In [2]:
# load local model and push to hugging_face_Hub
# learn = load_learner("E:\\GIT_REPOS\\Beetle_classifier\\Models\\beetle_classifier.pkl", cpu=False)

# # repo_id = "YOUR_USERNAME/YOUR_LEARNER_NAME"
# repo_id = "ChristopherMarais/Andrew_Alpha_model"
# push_to_hub_fastai(learner=learn, repo_id=repo_id)

In [3]:
# this function only describes how much a singular value in al ist stands out.
# if all values in the lsit are high or low this is 1
# the smaller the proportiopn of number of disimilar vlaues are to other more similar values the lower this number
# the larger the gap between the dissimilar numbers and the simialr number the smaller this number
# only able to interpret probabilities or values between 0 and 1
# this function outputs an estimate an inverse of the classification confidence based on the probabilities of all the classes.
# the wedge threshold splits the data on a threshold with a magnitude of a positive int to force a ledge/peak in the data
def unkown_prob_calc(probs, wedge_threshold, wedge_magnitude=1, wedge=True):
    if wedge:
        probs = np.where(probs<=wedge_threshold , probs**(2*wedge_magnitude), probs)
        probs = np.where(probs>=wedge_threshold , probs**(1/(2*wedge_magnitude)), probs)
    diff_matrix = np.abs(probs[:, np.newaxis] - probs)
    diff_matrix_sum = np.sum(diff_matrix)
    probs_sum = np.sum(probs)
    class_val = (diff_matrix_sum/probs_sum)
    max_class_val = ((len(probs)-1)*2)
    kown_prob = class_val/max_class_val
    unknown_prob = 1-kown_prob
    return(unknown_prob)

# load model
learn = load_learner("E:\\GIT_REPOS\\Beetle_classifier\\Models\\beetle_classifier.pkl", cpu=False)
# get class names
labels = np.append(np.array(learn.dls.vocab), "Unknown")

def predict(img):
    # Segment image into smaller images
    pre_process = pre_process_image(manual_thresh_buffer=0.15, image = img) # use image_dir if directory of image used
    pre_process.segment(cluster_num=2, 
                        image_edge_buffer=50)
    # get predictions for all segments
    conf_dict_lst = []
    output_lst = []
    img_cnt = len(pre_process.col_image_lst)
    for i in range(0,img_cnt):
        prob_ar = np.array(learn.predict(pre_process.col_image_lst[i])[2])
        unkown_prob = unkown_prob_calc(probs=prob_ar, wedge_threshold=0.9, wedge_magnitude=1, wedge=True)
        prob_ar = np.append(prob_ar, unkown_prob)
        conf_dict = {labels[i]: float(prob_ar[i]) for i in range(len(prob_ar))}
        conf_dict_lst.append(conf_dict)
        output_lst.append(conf_dict)
    return(pre_process.col_image_lst, conf_dict_lst)
    # return [gr.Image(pre_process.col_image_lst[0], type="numpy"), gr.Label(conf_dict_lst[0], num_top_classes=3)]

In [4]:
# wd = "F:\\Beetle_classification_deep_data\\Coccotypes_dactyliperda\\Vial_16296\\Subset_1"
# image_name = "IMG_0052.JPG"
# image_path = wd+"\\"+image_name
# image_arr = np.array(Image.open(image_path))
# a = predict(image_arr)

In [5]:
# launch app
gr.Interface(fn=predict, 
             inputs=gr.Image(type="numpy"),
             outputs=[gr.Image(type="numpy"), gr.Label(num_top_classes=3)],
             examples=["lion.jpg", "cheetah.jpg"]).launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


In [6]:
# # wd = "/blue/hulcr/gmarais/Beetle_data/Beetle_classification_deep_data/Coccotypes_dactyliperda/Vial_16296/Subset_1"
# wd = "F:\\Beetle_classification_deep_data\\Coccotypes_dactyliperda\\Vial_16296\\Subset_1"
# image_name = "IMG_0052.JPG"
# image_path = wd+"\\"+image_name


# ############################

# # load compound image
# # image_arr = io.imread(image_path)
# image_arr = np.array(Image.open(image_path))

In [7]:
# import torch
# import requests
# from PIL import Image
# from torchvision import transforms

In [8]:
# model = torch.hub.load('pytorch/vision:v0.6.0', 'resnet18', pretrained=True).eval()

# # Download human-readable labels for ImageNet.
# response = requests.get("https://git.io/JJkYN")
# labels = response.text.split("\n")

# def predict(inp_og):
#     inp = transforms.ToTensor()(inp_og).unsqueeze(0)
#     with torch.no_grad():
#         prediction = torch.nn.functional.softmax(model(inp)[0], dim=0)
#         confidences = {labels[i]: float(prediction[i]) for i in range(1000)}    
#     return [inp_og, confidences]

In [9]:
# import gradio as gr

# gr.Interface(fn=predict, 
#              inputs=gr.Image(type="pil"),
#              outputs=[gr.Image(type="pil"), gr.Label(num_top_classes=3)],
#              examples=["lion.jpg", "cheetah.jpg"]).launch()

C:\Users\GCM\anaconda3\envs\BC_310\lib\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


Traceback (most recent call last):
  File "C:\Users\GCM\anaconda3\envs\BC_310\lib\site-packages\gradio\routes.py", line 394, in run_predict
    output = await app.get_blocks().process_api(
  File "C:\Users\GCM\anaconda3\envs\BC_310\lib\site-packages\gradio\blocks.py", line 1078, in process_api
    data = self.postprocess_data(fn_index, result["prediction"], state)
  File "C:\Users\GCM\anaconda3\envs\BC_310\lib\site-packages\gradio\blocks.py", line 1012, in postprocess_data
    prediction_value = block.postprocess(prediction_value)
  File "C:\Users\GCM\anaconda3\envs\BC_310\lib\site-packages\gradio\components.py", line 1636, in postprocess
    raise ValueError("Cannot process this value as an Image")
ValueError: Cannot process this value as an Image
